In [1]:
cd /full/path/to/RefugeeFlowsAnalysis

[WinError 3] The system cannot find the path specified: '/full/path/to/RefugeeFlowsAnalysis'
C:\Users\odibs\RefugeeFlowsAnalysis


In [2]:
%cd C:\Users\odibs\RefugeeFlowsAnalysis

C:\Users\odibs\RefugeeFlowsAnalysis


In [4]:
git init

SyntaxError: invalid syntax (2830201818.py, line 1)

In [5]:
import os
os.getcwd()

'C:\\Users\\odibs\\RefugeeFlowsAnalysis'

In [6]:
import os
os.getcwd()

'C:\\Users\\odibs\\RefugeeFlowsAnalysis'

In [1]:
import pandas as pd

# Load the WDI dataset
df = pd.read_csv("data/wb/WDIData.csv")  # make sure the file is here!

# Keep only key columns and years 2010–2022
keep_cols = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'] + [str(y) for y in range(2010, 2023)]
slim_df = df[keep_cols]

# Save a slimmed version
slim_df.to_csv("data/wb/wdi_2010_2022.csv", index=False)

print("✅ Slim file saved as wdi_2010_2022.csv")


KeyError: "['2019', '2020', '2021', '2022'] not in index"

In [2]:
print(df.columns.tolist())

['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', 'Unnamed: 63']


In [3]:
import pandas as pd

# Step 1: Load the file
df = pd.read_csv("data/wb/WDIData.csv")

# Step 2: Keep valid columns (only up to 2018)
keep_cols = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'] + [str(y) for y in range(2010, 2019)]
slim_df = df[keep_cols]

# Step 3: Save the slimmed-down file
slim_df.to_csv("data/wb/wdi_2010_2018.csv", index=False)

print("✅ Slim file saved as wdi_2010_2018.csv")


✅ Slim file saved as wdi_2010_2018.csv
